## Predicting Dengue
## Muhammad Fuzail Zubari
## 18101135

In [215]:
import pandas as pd
import numpy as np
import collections
import seaborn as sns
import collections

import matplotlib.cm as cm
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt

from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf
import statsmodels.api as sm

from collections import Counter
from datetime import datetime, date, timedelta
from IPython.display import Image
%matplotlib inline

In [216]:
# Loading Data

dengue_features_train = pd.read_csv('dengue_features_train.csv')
dengue_features_test = pd.read_csv('dengue_features_test.csv')
dengue_labels_train = pd.read_csv('dengue_labels_train.csv')

In [217]:
dengue_features_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [218]:
dengue_features_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


### Data Preprocessing Test Data

In [219]:
dengue_features_test.isnull().sum()

city                                      0
year                                      0
weekofyear                                0
week_start_date                           0
ndvi_ne                                  43
ndvi_nw                                  11
ndvi_se                                   1
ndvi_sw                                   1
precipitation_amt_mm                      2
reanalysis_air_temp_k                     2
reanalysis_avg_temp_k                     2
reanalysis_dew_point_temp_k               2
reanalysis_max_air_temp_k                 2
reanalysis_min_air_temp_k                 2
reanalysis_precip_amt_kg_per_m2           2
reanalysis_relative_humidity_percent      2
reanalysis_sat_precip_amt_mm              2
reanalysis_specific_humidity_g_per_kg     2
reanalysis_tdtr_k                         2
station_avg_temp_c                       12
station_diur_temp_rng_c                  12
station_max_temp_c                        3
station_min_temp_c              

In [220]:
#Check duplicate rows
np.sum(dengue_features_test.duplicated())

0

In [221]:
dengue_features_test.fillna(method='ffill', inplace=True)

In [222]:
dengue_features_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,-0.012400,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,-0.0015,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [223]:
dengue_features_test.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

### Data Preprocessing Train Data

In [224]:
dengue_features_train = pd.merge(dengue_labels_train, dengue_features_train, on=['city','year','weekofyear'])

In [225]:
dengue_features_train.isnull().sum()

city                                       0
year                                       0
weekofyear                                 0
total_cases                                0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_ma

In [226]:
# Check for duplicated values
np.sum(dengue_features_train.duplicated())

0

In [227]:
# Forward or backward 'NaN' data filling
dengue_features_train.fillna(method='ffill', inplace=True)

In [228]:
dengue_features_train.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
total_cases                              0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min

### EDA

In [229]:
# Calculating the correlation
correlations = dengue_features_train.corr()

In [230]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

correlations.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,year,weekofyear,total_cases,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
year,1,-0.072,-0.31,0.2,0.15,0.25,0.28,0.21,-0.14,0.088,0.14,0.48,-0.39,0.13,0.34,0.21,0.17,0.49,0.05,0.38,0.23,-0.21,0.22
weekofyear,-0.072,1,0.22,0.024,0.035,0.1,0.064,0.12,0.42,0.46,0.35,0.23,0.18,0.072,-0.012,0.12,0.35,0.076,0.36,0.022,0.23,0.29,0.064
total_cases,-0.31,0.22,1,-0.21,-0.17,-0.21,-0.15,-0.043,0.26,0.15,0.14,-0.19,0.33,-0.012,-0.13,-0.043,0.13,-0.28,0.11,-0.24,-0.04,0.26,-0.074
ndvi_ne,0.2,0.024,-0.21,1,0.82,0.59,0.65,0.2,-0.33,-0.038,0.055,0.62,-0.61,0.19,0.46,0.2,0.092,0.67,0.18,0.66,0.48,-0.3,0.23
ndvi_nw,0.15,0.035,-0.17,0.82,1,0.55,0.65,0.19,-0.32,-0.032,0.064,0.61,-0.59,0.19,0.45,0.19,0.1,0.65,0.2,0.65,0.48,-0.3,0.21
ndvi_se,0.25,0.1,-0.21,0.59,0.55,1,0.81,0.084,-0.16,0.051,0.0056,0.47,-0.41,0.038,0.22,0.084,0.032,0.49,0.089,0.45,0.28,-0.25,0.13
ndvi_sw,0.28,0.064,-0.15,0.65,0.65,0.81,1,0.13,-0.23,0.024,0.025,0.55,-0.49,0.1,0.31,0.13,0.057,0.57,0.12,0.54,0.37,-0.29,0.16
precipitation_amt_mm,0.21,0.12,-0.043,0.2,0.19,0.084,0.13,1,-0.015,0.11,0.44,0.29,-0.12,0.48,0.5,1,0.45,0.2,0.23,0.19,0.29,0.081,0.48
reanalysis_air_temp_k,-0.14,0.42,0.26,-0.33,-0.32,-0.16,-0.23,-0.015,1,0.9,0.53,0.0018,0.74,-0.13,-0.4,-0.015,0.51,-0.28,0.61,-0.26,0.23,0.72,-0.17
reanalysis_avg_temp_k,0.088,0.46,0.15,-0.038,-0.032,0.051,0.024,0.11,0.9,1,0.61,0.4,0.44,-0.032,-0.17,0.11,0.61,0.13,0.74,0.11,0.51,0.58,-0.051


In [231]:
# Dropping columns with negative corellation in both 

columns_to_drop = ['reanalysis_tdtr_k', 'year', 'station_diur_temp_rng_c', 'ndvi_nw', 'weekofyear', 'ndvi_ne', 'ndvi_se', 'reanalysis_max_air_temp_k']

# Remove `week_start_date` string.
dengue_features_train.drop(columns_to_drop, axis=1, inplace=True)
dengue_features_test.drop(columns_to_drop, axis=1, inplace=True)


In [232]:
dengue_features_train.count()

city                                     1456
total_cases                              1456
week_start_date                          1456
ndvi_sw                                  1456
precipitation_amt_mm                     1456
reanalysis_air_temp_k                    1456
reanalysis_avg_temp_k                    1456
reanalysis_dew_point_temp_k              1456
reanalysis_min_air_temp_k                1456
reanalysis_precip_amt_kg_per_m2          1456
reanalysis_relative_humidity_percent     1456
reanalysis_sat_precip_amt_mm             1456
reanalysis_specific_humidity_g_per_kg    1456
station_avg_temp_c                       1456
station_max_temp_c                       1456
station_min_temp_c                       1456
station_precip_mm                        1456
dtype: int64

### Training and Testing of Data

In [233]:
subtrain = dengue_features_train
subtest = dengue_features_train.tail(dengue_features_train.shape[0] - 800)

In [238]:
def get_best_model(train, test):
    # Step 1: specify the form of the model
    model_formula = "total_cases ~ 1 + " \
                    "reanalysis_specific_humidity_g_per_kg + " \
                    "reanalysis_dew_point_temp_k + " \
                    "reanalysis_min_air_temp_k + " \
                    "station_min_temp_c + " \
                    "station_max_temp_c + " \
                    "station_avg_temp_c"
     
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)
                    
    best_alpha = []
    best_score = 1000
        
    # Step 2: Find the best hyper parameter, alpha
    for alpha in grid:
        model = smf.glm(formula=model_formula,
                        data=train,
                        family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(test).astype(int)
        score = eval_measures.meanabs(predictions, test.total_cases)

        if score < best_score:
            best_alpha = alpha
            best_score = score

    print('Alpha = ', best_alpha)
    print('Score = ', best_score)
            
    # Step 3: refit on entire dataset
    full_dataset = pd.concat([train, test])
    model = smf.glm(formula=model_formula,
                    data=full_dataset,
                    family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model

In [239]:
best_model = get_best_model(subtrain, subtest)
predictions = best_model.predict(dengue_features_test).astype(int)
#print(predictions)

Alpha =  1e-08
Score =  10.228658536585366


In [240]:
submission = pd.read_csv("submission_format.csv", index_col=[0, 1, 2])

submission.total_cases = np.concatenate([predictions])
submission.to_csv("predicted_values_2.0.csv")